In [ ]:
import json

result_file = "output/20250120-21:00-all-results.json"
with open(result_file,'r') as f:
    results = json.load(f)

results

In [ ]:
import matplotlib.pyplot as plt

key1 = 'maxStreak=3'
key2 = 'teams=50'
for key1 in [f"maxStreak={streakLen}" for streakLen in range(1, 7)]:
    for key2 in [f'teams={teamSize}' for teamSize in range(4, 52, 2)]:
        fig, axs = plt.subplots(nrows=1, ncols=3, sharey=True, constrained_layout=True)
        for i, key3 in enumerate(['doubleRoundRobinViolations', 'noRepeatViolations', 'maxStreakViolations']):
            histo = results[key1][key2][key3]
            # Extract values and labels from dictionary
            labels = sorted([int(c) for c in histo.keys()])
            values = [ histo[str(label)] for label in labels]
            axs[i].bar(labels, values, width=1)
            if i == 0:
                axs[i].set_ylabel('number of tournaments')
                repetitions = sum(results[key1][key2][key3].values())
            axs[i].set_title("".join(c.upper() for i,c in enumerate(key3) if c.isupper() or i == 0))
        fig.suptitle(f"{key1}, {key2}, evaluated tournaments={repetitions}")
        fig.supxlabel("number of violations per tournament")
        # plt.show()
        plt.savefig(f'figures/{key1.replace("=", "")}{key2.replace("=", "")}.png')

# 2025-01-11


In [3]:
def coefsToLatex(coefs):
    equation = ""
    l = len(coefs)
    for index, c in enumerate(coefs):
        n = (l - index) -1
        if round(c, 3) > 0:
            reprc = f"{'+' if index>0 else''}{round(c, 3):.3f}"
        else:
            reprc = f"{round(c, 3):.3f}"
        if n > 1:
            equation += f" {reprc}x^{n}"
        elif n == 1:
            equation += f" {reprc}x"
        else:
            equation += f" {reprc}"
    return equation

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.lines as mlines
import numpy as np

violations = ["maxStreakViolations", "noRepeatViolations", "doubleRoundRobinViolations"]
teamSizes = [i for i in range(4, 52, 2)]

def averageNumberViolations(data:dict[str,int]):
    keys  = [int(key) for key in data.keys()]
    return sum([key * data[str(key)] for key in keys])/sum(data.values())

def maximumNumberViolations(data:dict[str,int]):
    return max(map(int, data.keys()))

def minimumNumberViolations(data:dict[str,int]):
    return min(map(int, data.keys()))

for maxStreak in range(1, 7):
    
    alldrrv = []
    mindrrv = []
    maxdrrv = []

    allmsv = []
    minmsv = []
    maxmsv = []

    allnrv = []
    minnrv = []
    maxnrv = []

    for teamSize in teamSizes:
        currentData = results[f"maxStreak={maxStreak}"][f"teams={teamSize}"]

        alldrrv.append(averageNumberViolations(currentData["doubleRoundRobinViolations"]))
        mindrrv.append(minimumNumberViolations(currentData["doubleRoundRobinViolations"]))
        maxdrrv.append(maximumNumberViolations(currentData["doubleRoundRobinViolations"]))

        allmsv.append(averageNumberViolations(currentData["maxStreakViolations"]))
        minmsv.append(minimumNumberViolations(currentData["maxStreakViolations"]))
        maxmsv.append(maximumNumberViolations(currentData["maxStreakViolations"]))
    
        allnrv.append(averageNumberViolations(currentData["noRepeatViolations"]))
        minnrv.append(minimumNumberViolations(currentData["noRepeatViolations"]))
        maxnrv.append(maximumNumberViolations(currentData["noRepeatViolations"]))
    
    fig, axs = plt.subplots(ncols=2, figsize=(8, 4), sharey=False)

    ax = axs[0]
    
    ax.set_ylim(0,2000)
    # Plotting code
    ax.fill_between(teamSizes, mindrrv, maxdrrv, facecolor='salmon')  # 95% CI
    ax.plot(teamSizes, alldrrv, '-', label="All DRRV", color='black')  # All DRRV

    ax.fill_between(teamSizes, minmsv, maxmsv, facecolor='cornflowerblue')  # 95% CI
    ax.plot(teamSizes, allmsv, '-', label="All MSV", color='black')  # All MSV

    ax.fill_between(teamSizes, minnrv, maxnrv, facecolor='gold')  # 95% CI
    ax.plot(teamSizes, allnrv, '-', label="All NRV", color='black')  # All NRV

    # ax.set_xlabel("Number of Teams")
    fig.supxlabel("Number of Teams")
    ax.set_ylabel("Number of Violations")
    handles, labels = ax.get_legend_handles_labels()

    ax.legend([
        mpatches.Patch(color="salmon"),
        mpatches.Patch(color="cornflowerblue"),
        mpatches.Patch(color="gold"),
        mlines.Line2D([0], [0], color='black')
    ], 
    [
        "Double Round-Robin",
        "maxStreak",
        "noRepeat",
        "Average"
    ])

    drrvfit = np.polyfit(teamSizes, alldrrv, 2)
    drrvfit_fn = np.poly1d(drrvfit)
    # print("DRRV\n", drrvfit_fn)

    msvfit = np.polyfit(teamSizes, allmsv, 2)
    msvfit_fn = np.poly1d(msvfit) 
    # print("MSV\n", coefsToLatex(msvfit_fn.coef))

    nrvfit = np.polyfit(teamSizes, allnrv, 1)
    nrvfit_fn  = np.poly1d(nrvfit) 
    # print("NRV\n", coefsToLatex(nrvfit_fn.coef))

    ax = axs[1]
    xs = np.linspace(0,100)
    ys = drrvfit_fn(xs)
    ax.plot(xs, ys, "--", color='salmon',  label=rf"${coefsToLatex(drrvfit_fn.coef)}$")
    ys = msvfit_fn(xs)
    ax.plot(xs, ys, "--", color='cornflowerblue',  label=rf"${coefsToLatex(msvfit_fn.coef)}$")
    ys = nrvfit_fn(xs)
    ax.plot(xs, ys, "--", color='gold',  label=rf"${coefsToLatex(nrvfit_fn.coef)}$")
    ax.yaxis.set_label_position("right")
    ax.yaxis.tick_right()
    ax.legend()
    fig.title(f"maxStreak = {maxStreak}")
    fig.tight_layout()
    plt.savefig(f"figures/fitsForMaxStreak={maxStreak}.png")
    # plt.show()

# 2025-01-20

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.lines as mlines
import numpy as np

violations = ["maxStreakViolations", "noRepeatViolations", "doubleRoundRobinViolations"]
teamSizes = [i for i in range(4, 52, 2)]

def averageNumberViolations(data:dict[str,int]):
    keys  = [int(key) for key in data.keys()]
    return sum([key * data[str(key)] for key in keys])/sum(data.values())

def maximumNumberViolations(data:dict[str,int]):
    return max(map(int, data.keys()))

def minimumNumberViolations(data:dict[str,int]):
    return min(map(int, data.keys()))

fig, axs = plt.subplots(ncols=2, nrows=3, figsize=(10, 15), sharey=True, sharex=True, constrained_layout=True)

selector = 0
for maxStreak in range(1, 7):
    
    alldrrv = []
    mindrrv = []
    maxdrrv = []

    allmsv = []
    minmsv = []
    maxmsv = []

    allnrv = []
    minnrv = []
    maxnrv = []

    for teamSize in teamSizes:
        currentData = results[f"maxStreak={maxStreak}"][f"teams={teamSize}"]

        alldrrv.append(averageNumberViolations(currentData["doubleRoundRobinViolations"]))
        mindrrv.append(minimumNumberViolations(currentData["doubleRoundRobinViolations"]))
        maxdrrv.append(maximumNumberViolations(currentData["doubleRoundRobinViolations"]))

        allmsv.append(averageNumberViolations(currentData["maxStreakViolations"]))
        minmsv.append(minimumNumberViolations(currentData["maxStreakViolations"]))
        maxmsv.append(maximumNumberViolations(currentData["maxStreakViolations"]))
    
        allnrv.append(averageNumberViolations(currentData["noRepeatViolations"]))
        minnrv.append(minimumNumberViolations(currentData["noRepeatViolations"]))
        maxnrv.append(maximumNumberViolations(currentData["noRepeatViolations"]))

    if selector == 0:
        ax = axs[0][0]
        
        ax.set_ylim(0,2000)
        ax.set_xlim(0, 70)
        # Plotting code
        ax.fill_between(teamSizes, mindrrv, maxdrrv, facecolor='salmon')  # 95% CI
        ax.plot(teamSizes, alldrrv, '-', label="All DRRV", color='black')  # All DRRV

        ax.fill_between(teamSizes, minmsv, maxmsv, facecolor='cornflowerblue')  # 95% CI
        ax.plot(teamSizes, allmsv, '-', label="All MSV", color='black')  # All MSV

        ax.fill_between(teamSizes, minnrv, maxnrv, facecolor='gold')  # 95% CI
        ax.plot(teamSizes, allnrv, '-', label="All NRV", color='black')  # All NRV

        # ax.set_xlabel("Number of Teams")
        fig.supxlabel("Number of Teams")
        fig.supylabel("Number of Violations")
        handles, labels = ax.get_legend_handles_labels()

        ax.legend([
            mpatches.Patch(color="salmon"),
            mpatches.Patch(color="cornflowerblue"),
            mpatches.Patch(color="gold"),
            mlines.Line2D([0], [0], color='black')
        ], 
        [
            "Double Round-Robin",
            "maxStreak",
            "noRepeat",
            "Average"
        ])

    drrvfit = np.polyfit(teamSizes, alldrrv, 2)
    drrvfit_fn = np.poly1d(drrvfit)
    # print("DRRV\n", drrvfit_fn)

    msvfit = np.polyfit(teamSizes, allmsv, 2)
    msvfit_fn = np.poly1d(msvfit) 
    # print("MSV\n", coefsToLatex(msvfit_fn.coef))

    nrvfit = np.polyfit(teamSizes, allnrv, 1)
    nrvfit_fn  = np.poly1d(nrvfit) 
    # print("NRV\n", coefsToLatex(nrvfit_fn.coef))

    # selector += 1
    ax = axs[selector//2][selector%2]
    xs = np.linspace(0,100)
    ys = drrvfit_fn(xs)
    ax.plot(xs, ys, "--", color='salmon',  label=rf"${coefsToLatex(drrvfit_fn.coef)}$")
    ys = msvfit_fn(xs)
    ax.plot(xs, ys, "--", color='cornflowerblue',  label=rf"${coefsToLatex(msvfit_fn.coef)}$")
    ys = nrvfit_fn(xs)
    ax.plot(xs, ys, "--", color='gold',  label=rf"${coefsToLatex(nrvfit_fn.coef)}$")
    ax.yaxis.set_label_position("right")
    # ax.yaxis.tick_right()
    if selector > 0:
        ax.legend()
    # fig.suptitle(f"maxStreak = {maxStreak}")
    ax.set_title(f"maxStreak = {maxStreak}")
    fig.tight_layout()
    plt.savefig(f"figures/fitsForMaxStreak={maxStreak}.png")
    selector += 1
    # plt.show()

plt.savefig("figures/fits2Rows3Col.png")

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.lines as mlines
import numpy as np

violations = ["maxStreakViolations", "noRepeatViolations", "doubleRoundRobinViolations"]
teamSizes = [i for i in range(4, 52, 2)]

def averageNumberViolations(data:dict[str,int]):
    keys  = [int(key) for key in data.keys()]
    return sum([key * data[str(key)] for key in keys])/sum(data.values())

def maximumNumberViolations(data:dict[str,int]):
    return max(map(int, data.keys()))

def minimumNumberViolations(data:dict[str,int]):
    return min(map(int, data.keys()))

fig, axs = plt.subplots(ncols=2, nrows=3, figsize=(12, 18), sharey=True, sharex=True, constrained_layout=True)

selector = 0
for maxStreak in range(1, 7):
    
    alldrrv = []
    mindrrv = []
    maxdrrv = []

    allmsv = []
    minmsv = []
    maxmsv = []

    allnrv = []
    minnrv = []
    maxnrv = []

    for teamSize in teamSizes:
        currentData = results[f"maxStreak={maxStreak}"][f"teams={teamSize}"]

        alldrrv.append(averageNumberViolations(currentData["doubleRoundRobinViolations"]))
        mindrrv.append(minimumNumberViolations(currentData["doubleRoundRobinViolations"]))
        maxdrrv.append(maximumNumberViolations(currentData["doubleRoundRobinViolations"]))

        allmsv.append(averageNumberViolations(currentData["maxStreakViolations"]))
        minmsv.append(minimumNumberViolations(currentData["maxStreakViolations"]))
        maxmsv.append(maximumNumberViolations(currentData["maxStreakViolations"]))
    
        allnrv.append(averageNumberViolations(currentData["noRepeatViolations"]))
        minnrv.append(minimumNumberViolations(currentData["noRepeatViolations"]))
        maxnrv.append(maximumNumberViolations(currentData["noRepeatViolations"]))

    # if selector == 0:
    
    # ax = axs[0][0]
    ax = axs[selector//2][selector%2]
    ax.set_ylim(0,2000)
    ax.set_xlim(0, 70)
    # Plotting code
    ax.fill_between(teamSizes, mindrrv, maxdrrv, facecolor='salmon')  # 95% CI
    ax.plot(teamSizes, alldrrv, '-', label="All DRRV", color='black')  # All DRRV

    ax.fill_between(teamSizes, minmsv, maxmsv, facecolor='cornflowerblue')  # 95% CI
    ax.plot(teamSizes, allmsv, '-', label="All MSV", color='black')  # All MSV

    ax.fill_between(teamSizes, minnrv, maxnrv, facecolor='gold')  # 95% CI
    ax.plot(teamSizes, allnrv, '-', label="All NRV", color='black')  # All NRV

    # ax.set_xlabel("Number of Teams")
    fig.supxlabel("Number of Teams")
    fig.supylabel("Number of Violations")
    handles, labels = ax.get_legend_handles_labels()

    firstLegend = ax.legend([
        mpatches.Patch(color="salmon"),
        mpatches.Patch(color="cornflowerblue"),
        mpatches.Patch(color="gold"),
        mlines.Line2D([0], [0], color='black')
    ], 
    [
        "Double Round-Robin",
        "maxStreak",
        "noRepeat",
        "Average"
    ])

    # ax.set_ylabel("Number of Violations")
    drrvfit = np.polyfit(teamSizes, alldrrv, 2)
    drrvfit_fn = np.poly1d(drrvfit)
    # print("DRRV\n", drrvfit_fn)

    msvfit = np.polyfit(teamSizes, allmsv, 2)
    msvfit_fn = np.poly1d(msvfit) 
    # print("MSV\n", coefsToLatex(msvfit_fn.coef))

    nrvfit = np.polyfit(teamSizes, allnrv, 1)
    nrvfit_fn  = np.poly1d(nrvfit) 
    # print("NRV\n", coefsToLatex(nrvfit_fn.coef))

    # selector += 1
    xs = np.linspace(50,100)
    ys = drrvfit_fn(xs)
    fit1, = ax.plot(xs, ys, "--", color='salmon',  label=rf"${coefsToLatex(drrvfit_fn.coef)}$")
    ys = msvfit_fn(xs)
    fit2, = ax.plot(xs, ys, "--", color='cornflowerblue',  label=rf"${coefsToLatex(msvfit_fn.coef)}$")
    ys = nrvfit_fn(xs)
    fit3, = ax.plot(xs, ys, "--", color='gold',  label=rf"${coefsToLatex(nrvfit_fn.coef)}$")
    ax.yaxis.set_label_position("right")
    # ax.yaxis.tick_right()
    
    firstLegend = ax.legend([
        mpatches.Patch(color="salmon"),
        mpatches.Patch(color="cornflowerblue"),
        mpatches.Patch(color="gold"),
        mlines.Line2D([0], [0], color='black'),
        fit1, 
        fit2, 
        fit3
    ], 
    [
        "Double Round-Robin",
        "maxStreak",
        "noRepeat",
        "Average",
        rf"${coefsToLatex(drrvfit_fn.coef)}$",
        rf"${coefsToLatex(msvfit_fn.coef)}$",
        rf"${coefsToLatex(nrvfit_fn.coef)}$"
    ])
    ax.add_artist(firstLegend)
    # if selector > 0:
    # ax.legend(handles=[fit1, fit2, fit3], loc='center left')
    # fig.suptitle(f"maxStreak = {maxStreak}")
    # ax.set_title(f"maxStreak = {maxStreak}")
    # if selector != 1:
    ax.text(5, 1300, f"maxStreak = {maxStreak}", bbox={'facecolor':'lightblue'})
    # else:
        # ax.text(5,250, f"maxStreak = {maxStreak}")
    fig.tight_layout()
    plt.savefig(f"figures/fitsForMaxStreak={maxStreak}.png")
    selector += 1
    # plt.show()

plt.savefig("figures/fits3Rows2Col.png")

# 2025-01-25
## Are all DRRV even? 

In [12]:
for maxStreak in range(1, 6):
    for teams in range(4, 52, 2):
        data = results[f"maxStreak={maxStreak}"][f'teams={teams}']["doubleRoundRobinViolations"].keys()
        assert all(map(lambda x : int(x) % 2 == 0, data)), data

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.lines as mlines
import numpy as np

violations = ["maxStreakViolations", "noRepeatViolations", "doubleRoundRobinViolations"]
teamSizes = [i for i in range(4, 52, 2)]

def averageNumberViolations(data:dict[str,int]):
    keys  = [int(key) for key in data.keys()]
    return sum([key * data[str(key)] for key in keys])/sum(data.values())

def maximumNumberViolations(data:dict[str,int]):
    return max(map(int, data.keys()))

def minimumNumberViolations(data:dict[str,int]):
    return min(map(int, data.keys()))

fig, axs = plt.subplots(ncols=2, nrows=3, figsize=(12, 18), sharey=True, sharex=True, constrained_layout=True)

selector = 0
for maxStreak in range(1, 7):
    
    alldrrv = []
    mindrrv = []
    maxdrrv = []

    allmsv = []
    minmsv = []
    maxmsv = []

    allnrv = []
    minnrv = []
    maxnrv = []

    for teamSize in teamSizes:
        currentData = results[f"maxStreak={maxStreak}"][f"teams={teamSize}"]

        alldrrv.append(averageNumberViolations(currentData["doubleRoundRobinViolations"]))
        mindrrv.append(minimumNumberViolations(currentData["doubleRoundRobinViolations"]))
        maxdrrv.append(maximumNumberViolations(currentData["doubleRoundRobinViolations"]))

        allmsv.append(averageNumberViolations(currentData["maxStreakViolations"]))
        minmsv.append(minimumNumberViolations(currentData["maxStreakViolations"]))
        maxmsv.append(maximumNumberViolations(currentData["maxStreakViolations"]))
    
        allnrv.append(averageNumberViolations(currentData["noRepeatViolations"]))
        minnrv.append(minimumNumberViolations(currentData["noRepeatViolations"]))
        maxnrv.append(maximumNumberViolations(currentData["noRepeatViolations"]))

    # if selector == 0:
    
    # ax = axs[0][0]
    ax = axs[selector//2][selector%2]
    ax.set_ylim(0,2000)
    ax.set_xlim(0, 70)
    # Plotting code
    ax.fill_between(teamSizes, mindrrv, maxdrrv, facecolor='salmon')  # 95% CI
    ax.plot(teamSizes, alldrrv, '-', label="All DRRV", color='black')  # All DRRV

    ax.fill_between(teamSizes, minmsv, maxmsv, facecolor='cornflowerblue')  # 95% CI
    ax.plot(teamSizes, allmsv, '-', label="All MSV", color='black')  # All MSV

    ax.fill_between(teamSizes, minnrv, maxnrv, facecolor='gold')  # 95% CI
    ax.plot(teamSizes, allnrv, '-', label="All NRV", color='black')  # All NRV

    # ax.set_xlabel("Number of Teams")
    fig.supxlabel("Number of Teams")
    fig.supylabel("Number of Violations")
    handles, labels = ax.get_legend_handles_labels()


    # ax.set_ylabel("Number of Violations")
    drrvfit = np.polyfit(teamSizes, alldrrv, 2)
    drrvfit_fn = np.poly1d(drrvfit)
    # print("DRRV\n", drrvfit_fn)

    msvfit = np.polyfit(teamSizes, allmsv, 2)
    msvfit_fn = np.poly1d(msvfit) 
    # print("MSV\n", coefsToLatex(msvfit_fn.coef))

    nrvfit = np.polyfit(teamSizes, allnrv, 1)
    nrvfit_fn  = np.poly1d(nrvfit) 
    # print("NRV\n", coefsToLatex(nrvfit_fn.coef))

    # selector += 1
    xs = np.linspace(50,100)
    ys = drrvfit_fn(xs)
    fit1, = ax.plot(xs, ys, "--", color='salmon',  label=rf"${coefsToLatex(drrvfit_fn.coef)}$")
    ys = msvfit_fn(xs)
    fit2, = ax.plot(xs, ys, "--", color='cornflowerblue',  label=rf"${coefsToLatex(msvfit_fn.coef)}$")
    ys = nrvfit_fn(xs)
    fit3, = ax.plot(xs, ys, "--", color='gold',  label=rf"${coefsToLatex(nrvfit_fn.coef)}$")
    ax.yaxis.set_label_position("right")
    # ax.yaxis.tick_right()
    
    if selector == 0:
        firstLegend = ax.legend([
            mpatches.Patch(color="salmon"),
            mpatches.Patch(color="cornflowerblue"),
            mpatches.Patch(color="gold"),
            mlines.Line2D([0], [0], color='black'),
            # fit1, 
            # fit2, 
            # fit3
        ], 
        [
            "Double Round-Robin",
            "maxStreak",
            "noRepeat",
            "Average",
            # rf"${coefsToLatex(drrvfit_fn.coef)}$",
            # rf"${coefsToLatex(msvfit_fn.coef)}$",
            # rf"${coefsToLatex(nrvfit_fn.coef)}$"
        ])
        ax.add_artist(firstLegend)
    # if selector > 0:
    # ax.legend(handles=[fit1, fit2, fit3], loc='center left')
    # fig.suptitle(f"maxStreak = {maxStreak}")
    # ax.set_title(f"maxStreak = {maxStreak}")
    if selector < 1:
        ax.text(5, 1500, f"maxStreak = {maxStreak}", bbox={'facecolor':'white'})
    else:
        ax.text(5, 1800, f"maxStreak = {maxStreak}", bbox={'facecolor':'white'})

    if selector != 0:
        ax.text(5, 1100, rf"${coefsToLatex(drrvfit_fn.coef)}$", bbox={'facecolor':'salmon'})
    else:
        ax.text(43, 1100, rf"${coefsToLatex(drrvfit_fn.coef)}$", bbox={'facecolor':'salmon'})
    
    if selector == 0:
        ax.text(5, 1100, rf"${coefsToLatex(msvfit_fn.coef)}$", bbox={'facecolor':'cornflowerblue'})
    elif selector == 1:
        ax.text(40,  500, rf"${coefsToLatex(msvfit_fn.coef)}$", bbox={'facecolor':'cornflowerblue'})
    elif selector == 2:
        ax.text(5, 1000, rf"${coefsToLatex(msvfit_fn.coef)}$", bbox={'facecolor':'cornflowerblue'})
    # elif selector == 3:
    #     ax.text(50, 750, rf"${coefsToLatex(msvfit_fn.coef)}$", bbox={'facecolor':'cornflowerblue'})
    else:
        ax.text(32, 500, rf"${coefsToLatex(msvfit_fn.coef)}$", bbox={'facecolor':'cornflowerblue'})
    
    if selector != 3:
        ax.text(50, 300, rf"${coefsToLatex(nrvfit_fn.coef)}$", bbox={'facecolor':'gold'})
    else:
        ax.text(52, 150, rf"${coefsToLatex(nrvfit_fn.coef)}$", bbox={'facecolor':'gold'})

    # else:
        # ax.text(5,250, f"maxStreak = {maxStreak}")
    fig.tight_layout()
    plt.savefig(f"figures/fitsForMaxStreak={maxStreak}.png")
    selector += 1
    # plt.show()

plt.savefig("figures/fits3Rows2Col.png")

# 2025-01-26

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.lines as mlines
import numpy as np

violations = ["maxStreakViolations", "noRepeatViolations", "doubleRoundRobinViolations"]
teamSizes = [i for i in range(4, 52, 2)]

def averageNumberViolations(data:dict[str,int]):
    keys  = [int(key) for key in data.keys()]
    return sum([key * data[str(key)] for key in keys])/sum(data.values())

def maximumNumberViolations(data:dict[str,int]):
    return max(map(int, data.keys()))

def minimumNumberViolations(data:dict[str,int]):
    return min(map(int, data.keys()))

fig, axs = plt.subplots(ncols=3, nrows=2, figsize=(15, 10), sharey=True, sharex=True, constrained_layout=True)

selector = 0
for maxStreak in range(1, 7):
    
    alldrrv = []
    mindrrv = []
    maxdrrv = []

    allmsv = []
    minmsv = []
    maxmsv = []

    allnrv = []
    minnrv = []
    maxnrv = []

    for teamSize in teamSizes:
        currentData = results[f"maxStreak={maxStreak}"][f"teams={teamSize}"]

        alldrrv.append(averageNumberViolations(currentData["doubleRoundRobinViolations"]))
        mindrrv.append(minimumNumberViolations(currentData["doubleRoundRobinViolations"]))
        maxdrrv.append(maximumNumberViolations(currentData["doubleRoundRobinViolations"]))

        allmsv.append(averageNumberViolations(currentData["maxStreakViolations"]))
        minmsv.append(minimumNumberViolations(currentData["maxStreakViolations"]))
        maxmsv.append(maximumNumberViolations(currentData["maxStreakViolations"]))
    
        allnrv.append(averageNumberViolations(currentData["noRepeatViolations"]))
        minnrv.append(minimumNumberViolations(currentData["noRepeatViolations"]))
        maxnrv.append(maximumNumberViolations(currentData["noRepeatViolations"]))

    # if selector == 0:
    
    # ax = axs[0][0]
    ax = axs[selector//3][selector%3]
    ax.set_ylim(0,2000)
    ax.set_xlim(0, 50)
    # Plotting code
    ax.fill_between(teamSizes, mindrrv, maxdrrv, facecolor='salmon')  # 95% CI
    ax.plot(teamSizes, alldrrv, '-', label="All DRRV", color='black')  # All DRRV

    ax.fill_between(teamSizes, minmsv, maxmsv, facecolor='cornflowerblue')  # 95% CI
    ax.plot(teamSizes, allmsv, '-', label="All MSV", color='black')  # All MSV

    ax.fill_between(teamSizes, minnrv, maxnrv, facecolor='gold')  # 95% CI
    ax.plot(teamSizes, allnrv, '-', label="All NRV", color='black')  # All NRV

    # ax.set_xlabel("Number of Teams")
    fig.supxlabel("Number of Teams")
    fig.supylabel("Number of Violations")
    handles, labels = ax.get_legend_handles_labels()


    # ax.set_ylabel("Number of Violations")
    drrvfit = np.polyfit(teamSizes, alldrrv, 2)
    drrvfit_fn = np.poly1d(drrvfit)
    # print("DRRV\n", drrvfit_fn)

    msvfit = np.polyfit(teamSizes, allmsv, 2)
    msvfit_fn = np.poly1d(msvfit) 
    # print("MSV\n", coefsToLatex(msvfit_fn.coef))

    nrvfit = np.polyfit(teamSizes, allnrv, 1)
    nrvfit_fn  = np.poly1d(nrvfit) 
    # print("NRV\n", coefsToLatex(nrvfit_fn.coef))

    # selector += 1
    xs = np.linspace(50,100)
    ys = drrvfit_fn(xs)
    fit1, = ax.plot(xs, ys, "--", color='salmon',  label=rf"${coefsToLatex(drrvfit_fn.coef)}$")
    ys = msvfit_fn(xs)
    fit2, = ax.plot(xs, ys, "--", color='cornflowerblue',  label=rf"${coefsToLatex(msvfit_fn.coef)}$")
    ys = nrvfit_fn(xs)
    fit3, = ax.plot(xs, ys, "--", color='gold',  label=rf"${coefsToLatex(nrvfit_fn.coef)}$")
    ax.yaxis.set_label_position("right")
    # ax.yaxis.tick_right()
    
    if selector == 0:
        firstLegend = ax.legend([
            mpatches.Patch(color="salmon"),
            mpatches.Patch(color="cornflowerblue"),
            mpatches.Patch(color="gold"),
            mlines.Line2D([0], [0], color='black'),
            # fit1, 
            # fit2, 
            # fit3
        ], 
        [
            "Double Round-Robin",
            "maxStreak",
            "noRepeat",
            "Average",
            # rf"${coefsToLatex(drrvfit_fn.coef)}$",
            # rf"${coefsToLatex(msvfit_fn.coef)}$",
            # rf"${coefsToLatex(nrvfit_fn.coef)}$"
        ])
        ax.add_artist(firstLegend)
    # if selector > 0:
    # ax.legend(handles=[fit1, fit2, fit3], loc='center left')
    # fig.suptitle(f"maxStreak = {maxStreak}")
    # ax.set_title(f"maxStreak = {maxStreak}")
    if selector < 1:
        ax.text(20, 1400, f"maxStreak = {maxStreak}", bbox={'facecolor':'lightgrey'})
    else:
        ax.text(20, 1800, f"maxStreak = {maxStreak}", bbox={'facecolor':'lightgrey'})

    # if selector == 0:
    #     ax.text(30, 500, rf"${coefsToLatex(drrvfit_fn.coef)}$", bbox={'facecolor':'salmon'})
    # else:
    #     ax.text(5, 750, rf"${coefsToLatex(drrvfit_fn.coef)}$", bbox={'facecolor':'salmon'})
    ax.text(5, 1040, rf"${coefsToLatex(drrvfit_fn.coef)}$", bbox={'facecolor':'salmon'})
    
    # if selector == 0:
    #     ax.text(3, 750, rf"${coefsToLatex(msvfit_fn.coef)}$", bbox={'facecolor':'cornflowerblue'})
    # elif selector == 1:
    #     ax.text(30, 250, rf"${coefsToLatex(msvfit_fn.coef)}$", bbox={'facecolor':'cornflowerblue'})
    # elif selector == 2:
    #     ax.text(5, 500, rf"${coefsToLatex(msvfit_fn.coef)}$", bbox={'facecolor':'cornflowerblue'})
    # elif selector > 2:
    #     ax.text(27, 300, rf"${coefsToLatex(msvfit_fn.coef)}$", bbox={'facecolor':'cornflowerblue'})
    ax.text(5, 1200, rf"${coefsToLatex(msvfit_fn.coef)}$", bbox={'facecolor':'cornflowerblue'})
    
    # if selector < 3:
    #     ax.text(30, 110, rf"${coefsToLatex(nrvfit_fn.coef)}$", bbox={'facecolor':'gold'})
    # else:
    #     ax.text(5, 150, rf"${coefsToLatex(nrvfit_fn.coef)}$", bbox={'facecolor':'gold'})
    ax.text(5, 900, rf"${coefsToLatex(nrvfit_fn.coef)}$", bbox={'facecolor':'gold'})

    # else:
        # ax.text(5,250, f"maxStreak = {maxStreak}")
    fig.tight_layout()
    plt.savefig(f"figures/fitsForMaxStreak={maxStreak}.png")
    selector += 1
    # plt.show()

# plt.savefig("figures/fits2Rows3Col.png")

# 2025-01-27

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.lines as mlines
import numpy as np

violations = ["maxStreakViolations", "noRepeatViolations", "doubleRoundRobinViolations"]
teamSizes = [i for i in range(4, 52, 2)]

def averageNumberViolations(data:dict[str,int]):
    keys  = [int(key) for key in data.keys()]
    return sum([key * data[str(key)] for key in keys])/sum(data.values())

def maximumNumberViolations(data:dict[str,int]):
    return max(map(int, data.keys()))

def minimumNumberViolations(data:dict[str,int]):
    return min(map(int, data.keys()))

fig, axs = plt.subplots(ncols=3, nrows=2, figsize=(15, 10), sharey=True, sharex=True, constrained_layout=True)

selector = 0
for maxStreak in range(1, 7):
    
    alldrrv = []
    mindrrv = []
    maxdrrv = []

    allmsv = []
    minmsv = []
    maxmsv = []

    allnrv = []
    minnrv = []
    maxnrv = []

    for teamSize in teamSizes:
        currentData = results[f"maxStreak={maxStreak}"][f"teams={teamSize}"]

        alldrrv.append(averageNumberViolations(currentData["doubleRoundRobinViolations"]))
        mindrrv.append(minimumNumberViolations(currentData["doubleRoundRobinViolations"]))
        maxdrrv.append(maximumNumberViolations(currentData["doubleRoundRobinViolations"]))

        allmsv.append(averageNumberViolations(currentData["maxStreakViolations"]))
        minmsv.append(minimumNumberViolations(currentData["maxStreakViolations"]))
        maxmsv.append(maximumNumberViolations(currentData["maxStreakViolations"]))
    
        allnrv.append(averageNumberViolations(currentData["noRepeatViolations"]))
        minnrv.append(minimumNumberViolations(currentData["noRepeatViolations"]))
        maxnrv.append(maximumNumberViolations(currentData["noRepeatViolations"]))

    # if selector == 0:
    
    # ax = axs[0][0]
    ax = axs[selector//3][selector%3]
    ax.set_ylim(0,2000)
    ax.set_xlim(0, 50)
    # Plotting code
    ax.fill_between(teamSizes, mindrrv, maxdrrv, facecolor='salmon')  # 95% CI
    ax.plot(teamSizes, alldrrv, '-', label="All DRRV", color='black')  # All DRRV

    ax.fill_between(teamSizes, minmsv, maxmsv, facecolor='cornflowerblue')  # 95% CI
    ax.plot(teamSizes, allmsv, '-', label="All MSV", color='black')  # All MSV

    ax.fill_between(teamSizes, minnrv, maxnrv, facecolor='gold')  # 95% CI
    ax.plot(teamSizes, allnrv, '-', label="All NRV", color='black')  # All NRV

    # ax.set_xlabel("Number of Teams")
    fig.supxlabel("Number of Teams")
    fig.supylabel("Number of Violations")
    handles, labels = ax.get_legend_handles_labels()


    # ax.set_ylabel("Number of Violations")
    drrvfit = np.polyfit(teamSizes, alldrrv, 2)
    drrvfit_fn = np.poly1d(drrvfit)
    # print("DRRV\n", drrvfit_fn)

    msvfit = np.polyfit(teamSizes, allmsv, 2)
    msvfit_fn = np.poly1d(msvfit) 
    # print("MSV\n", coefsToLatex(msvfit_fn.coef))

    nrvfit = np.polyfit(teamSizes, allnrv, 1)
    nrvfit_fn  = np.poly1d(nrvfit) 
    # print("NRV\n", coefsToLatex(nrvfit_fn.coef))

    # selector += 1
    xs = np.linspace(50,100)
    ys = drrvfit_fn(xs)
    fit1, = ax.plot(xs, ys, "--", color='salmon',  label=rf"${coefsToLatex(drrvfit_fn.coef)}$")
    ys = msvfit_fn(xs)
    fit2, = ax.plot(xs, ys, "--", color='cornflowerblue',  label=rf"${coefsToLatex(msvfit_fn.coef)}$")
    ys = nrvfit_fn(xs)
    fit3, = ax.plot(xs, ys, "--", color='gold',  label=rf"${coefsToLatex(nrvfit_fn.coef)}$")
    ax.yaxis.set_label_position("right")
    # ax.yaxis.tick_right()
    
    if selector == 0:
        firstLegend = ax.legend([
            mpatches.Patch(color="salmon"),
            mpatches.Patch(color="cornflowerblue"),
            mpatches.Patch(color="gold"),
            mlines.Line2D([0], [0], color='black'),
            # fit1, 
            # fit2, 
            # fit3
        ], 
        [
            "Double Round-Robin",
            "maxStreak",
            "noRepeat",
            "Average",
            # rf"${coefsToLatex(drrvfit_fn.coef)}$",
            # rf"${coefsToLatex(msvfit_fn.coef)}$",
            # rf"${coefsToLatex(nrvfit_fn.coef)}$"
        ], loc=(0.05, 0.65))#, prop={'weight':'bold'})
        # ax.add_artist(firstLegend)
    # if selector > 0:
    # ax.legend(handles=[fit1, fit2, fit3], loc='center left')
    # fig.suptitle(f"maxStreak = {maxStreak}")
    # ax.set_title(f"maxStreak = {maxStreak}")
    # if selector < 1:
    #     ax.text(20, 1400, f"maxStreak = {maxStreak}", bbox={'facecolor':'lightgrey'})
    # else:
    ax.text(20, 1800, f"maxStreak = {maxStreak}", bbox={'facecolor':'lightgrey'}, fontweight="bold")

    if selector == 0:
        ax.text(27, 500, rf"${coefsToLatex(drrvfit_fn.coef)}$", bbox={'facecolor':'salmon'})
    else:
        ax.text(25, 1500, rf"${coefsToLatex(drrvfit_fn.coef)}$", bbox={'facecolor':'salmon'})
    # ax.text(5, 1040, rf"${coefsToLatex(drrvfit_fn.coef)}$", bbox={'facecolor':'salmon'})
    
    # if selector == 0:
    #     ax.text(3, 750, rf"${coefsToLatex(msvfit_fn.coef)}$", bbox={'facecolor':'cornflowerblue'})
    # elif selector == 1:
    #     ax.text(30, 250, rf"${coefsToLatex(msvfit_fn.coef)}$", bbox={'facecolor':'cornflowerblue'})
    # elif selector == 2:
    #     ax.text(5, 500, rf"${coefsToLatex(msvfit_fn.coef)}$", bbox={'facecolor':'cornflowerblue'})
    # elif selector > 2:
    #     ax.text(27, 300, rf"${coefsToLatex(msvfit_fn.coef)}$", bbox={'facecolor':'cornflowerblue'})
    ax.text(2, 400, rf"${coefsToLatex(msvfit_fn.coef)}$", bbox={'facecolor':'cornflowerblue'})
    
    if selector < 3:
        ax.text(30, 110, rf"${coefsToLatex(nrvfit_fn.coef)}$", bbox={'facecolor':'gold'})
    else:
        ax.text(30, 400, rf"${coefsToLatex(nrvfit_fn.coef)}$", bbox={'facecolor':'gold'})
    # ax.text(5, 900, rf"${coefsToLatex(nrvfit_fn.coef)}$", bbox={'facecolor':'gold'})

    # else:
        # ax.text(5,250, f"maxStreak = {maxStreak}")
    fig.tight_layout()
    plt.savefig(f"figures/fitsForMaxStreak={maxStreak}.png")
    selector += 1
    # plt.show()

plt.savefig("figures/fits2Rows3Col.png")